In [1]:
from pprint import pprint
import re

from datasets import load_dataset, Dataset, DatasetDict
import jsonlines as jsonl

In [2]:
dataset = load_dataset("LDJnr/Verified-Camel")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
dataset = dataset["train"].map(lambda x: dict(
    input=x["conversation"][0]["input"],
    output=x["conversation"][0]["output"],
)).remove_columns(["conversation", "source"])

In [9]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)


In [10]:
def to_chatml(row):
    input = row["input"]
    output = row["output"]
    newline = "\n"
    
    system_message = (
        "An AI assistant is helping a user with a scientific question."
    )

    # Turn into chatml
    chatml = [
        situation(system_message),
        person(input, name="User"),
        me(output, name="AI Assistant"),
    ]

    return dict(chatml=chatml)


In [11]:
dataset = dataset.map(to_chatml)

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

In [12]:
dataset = dataset.remove_columns(["input", "output"])

In [13]:
dataset.push_to_hub("diwank/verified-camel-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]